<a href="https://colab.research.google.com/github/batchuraja484/NLP_Tf-Keras/blob/main/IMDB_Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  7888k      0  0:00:10  0:00:10 --:--:-- 17.6M


In [3]:
!ls aclImdb  

imdbEr.txt  imdb.vocab	README	test  train


In [4]:
!cat aclImdb/train/pos/6248_7.txt

Being an Austrian myself this has been a straight knock in my face. Fortunately I don't live nowhere near the place where this movie takes place but unfortunately it portrays everything that the rest of Austria hates about Viennese people (or people close to that region). And it is very easy to read that this is exactly the directors intention: to let your head sink into your hands and say "Oh my god, how can THAT be possible!". No, not with me, the (in my opinion) totally exaggerated uncensored swinger club scene is not necessary, I watch porn, sure, but in this context I was rather disgusted than put in the right context.<br /><br />This movie tells a story about how misled people who suffer from lack of education or bad company try to survive and live in a world of redundancy and boring horizons. A girl who is treated like a whore by her super-jealous boyfriend (and still keeps coming back), a female teacher who discovers her masochism by putting the life of her super-cruel "lover" 

Remove unnecessary files

In [5]:
!rm -r aclImdb/train/unsup

In [6]:
batch_size=32
raw_train_ds=tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/train",
    subset="training",
    batch_size=batch_size,
    seed=1337,
    validation_split=0.2
)

raw_validation_ds=tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    seed=1337,
    subset="validation",
    validation_split=0.2
)
raw_test_ds=tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/test",
    batch_size=batch_size
)



print("Number of batches in raw_train_dataset :%d" %tf.data.experimental.cardinality(raw_train_ds))
print("NUmber of batchs in validation_raw_dataset :%d" %tf.data.experimental.cardinality(raw_validation_ds))
print("NUmber of batchs in Test_raw_dataset :%d" %tf.data.experimental.cardinality(raw_test_ds))

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.
Number of batches in raw_train_dataset :625
NUmber of batchs in validation_raw_dataset :157
NUmber of batchs in Test_raw_dataset :782


In [7]:
for text,label in raw_train_ds.take(1):
  for i in range(5):
    print(text.numpy()[i])
    print(label.numpy()[i])

b'I\'ve seen tons of science fiction from the 70s; some horrendously bad, and others thought provoking and truly frightening. Soylent Green fits into the latter category. Yes, at times it\'s a little campy, and yes, the furniture is good for a giggle or two, but some of the film seems awfully prescient. Here we have a film, 9 years before Blade Runner, that dares to imagine the future as somthing dark, scary, and nihilistic. Both Charlton Heston and Edward G. Robinson fare far better in this than The Ten Commandments, and Robinson\'s assisted-suicide scene is creepily prescient of Kevorkian and his ilk. Some of the attitudes are dated (can you imagine a filmmaker getting away with the "women as furniture" concept in our oh-so-politically-correct-90s?), but it\'s rare to find a film from the Me Decade that actually can make you think. This is one I\'d love to see on the big screen, because even in a widescreen presentation, I don\'t think the overall scope of this film would receive its

Prepare Data

In [13]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import string
import re

def custom_standardication(input_data):
  lowercase=tf.strings.lower(input_data)
  stripped_html=tf.strings.regex_replace(lowercase,"<br />"," ")
  return tf.strings.regex_replace(
      stripped_html,"[%s]" % re.escape(string.punctuation),""
  )

In [15]:
#model constraints

max_features=2000
embeding_dim=128
sequence_length=500

vectorize_layer=TextVectorization(
    standardize=custom_standardication,
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)

In [16]:
text_ds=raw_train_ds.map(lambda x,y:x)
vectorize_layer.adapt(text_ds)

In [17]:
def vectorize_text(text,label):
  text=tf.expand_dims(text,-1)
  return vectorize_layer(text),label

train_ds=raw_test_ds.map(vectorize_text)
val_ds=raw_validation_ds.map(vectorize_text)
test_ds=raw_test_ds.map(vectorize_text)

trains_ds=train_ds.cache().prefetch(buffer_size=10)
val_ds=val_ds.cache().prefetch(buffer_size=10)
text_ds=test_ds.cache().prefetch(buffer_size=10)

In [23]:
#build model
from tensorflow.keras import layers

inputs=tf.keras.Input(shape=(None,),dtype="int64")
x=layers.Embedding(max_features,embeding_dim)(inputs)
x=layers.Dropout(0.5)(x)

x=layers.Conv1D(128,7,padding='valid',activation="relu",strides=3)(x)
x=layers.Conv1D(128,7,padding='valid',activation='relu',strides=3)(x)
x=layers.GlobalMaxPooling1D()(x)
x=layers.Dense(128,activation='relu')(x)
x=layers.Dropout(0.5)(x)
predictions=layers.Dense(1,activation="sigmoid",name="predictions")(x)
model=tf.keras.Model(inputs,predictions)
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [24]:
train_epochs=5
model.fit(train_ds,validation_data=val_ds,epochs=train_epochs)

Epoch 1/5
782/782 [==============================] - 134s 171ms/step - loss: 0.4655 - accuracy: 0.7532 - val_loss: 0.4007 - val_accuracy: 0.8308
Epoch 2/5
782/782 [==============================] - 131s 167ms/step - loss: 0.2898 - accuracy: 0.8798 - val_loss: 0.3521 - val_accuracy: 0.8574
Epoch 3/5
782/782 [==============================] - 131s 167ms/step - loss: 0.2332 - accuracy: 0.9075 - val_loss: 0.3635 - val_accuracy: 0.8616
Epoch 4/5
782/782 [==============================] - 131s 168ms/step - loss: 0.1948 - accuracy: 0.9231 - val_loss: 0.3927 - val_accuracy: 0.8572
Epoch 5/5
782/782 [==============================] - 130s 166ms/step - loss: 0.1533 - accuracy: 0.9400 - val_loss: 0.4183 - val_accuracy: 0.8620


In [25]:
model.evaluate(test_ds)

782/782 [==============================] - 33s 43ms/step - loss: 0.0639 - accuracy: 0.9804


[0.06394892185926437, 0.9804400205612183]

In [34]:
test_ds.element_spec

(TensorSpec(shape=(None, 500), dtype=tf.int64, name=None),
 TensorSpec(shape=(None,), dtype=tf.int32, name=None))

In [35]:
predict_text="Movie is boring and slow"
predict_text

'Movie is boring and slow'